In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from keras.optimizers import SGD
from keras.preprocessing import image
import numpy as np

# Dimensions of our images.
img_width, img_height = 150, 150  #lowering computations

train_data_dir = '/content/drive/MyDrive/images/dataimages'
validation_data_dir = '/content/drive/MyDrive/images/Validation'
test_data_dir = '/content/drive/MyDrive/images/test'
nb_train_samples = 177 # 60
nb_validation_samples = 114 # 40
epochs = 20
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Load pretrained VGG16 model.
# The last (top) layers doing the final classification are not included.
vgg16 = VGG16(weights='imagenet', input_shape=input_shape, include_top=False)

# Freeze the weights for the first layers.
for layer in vgg16.layers[:17]:
    layer.trainable = False
# vgg16.summary()

# Add custom layers.
x = vgg16.output
x = Flatten()(x)
x = Dense(units=64, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(units=3, activation="softmax")(x)
print(vgg16.input)
# Create final model.
#model = Model(inputs = vgg16.input, output = predictions)
model = Model(inputs=vgg16.input, outputs=predictions)
model.compile(loss='CategoricalCrossentropy',
              optimizer=SGD(lr=0.001, momentum=0.9),
              metrics=['accuracy'])
history = model.summary()

tf.keras.utils.plot_model(
  model, to_file='/content/model.png', show_shapes=True, show_dtype=False,
  show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
  )

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

print(test_generator)
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)



KerasTensor(type_spec=TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Found 177 images belonging to 3 classes.
Found 114 images belonging to 3 classes.
Found 9 images belonging to 3 classes.
Epoch 1/20
5/5 [==============================] - 8s 1s/step - loss: 1.2445 - accuracy: 0.3784 - val_loss: 1.1218 - val_accuracy: 0.3125
Epoch 2/20
5/5 [==============================] - 5s 1s/step - loss: 1.1114 - accuracy: 0.4484 - val_loss: 1.0865 - val_accuracy: 0.4479
Epoch 3/20
5/5 [==============================] - 5s 1s/step - loss: 0.9065 - accuracy: 0.5700 - val_loss: 1.0763 - val_accuracy: 0.3750
Epoch 4/20
5/5 [==============================] - 5s 1s/step - loss: 0.7557 - accuracy: 0.7016 - val_loss: 1.0333 - val_accuracy: 0.3750
Epoch 5/20
5/5 [==============================] - 5s 1s/step - loss: 0.6828 - accuracy: 0.7107 - val_loss: 1.0648 - val_accuracy: 0.3438
Epoch 6/20
5/5 [==============================] - 5s 1s/step - loss: 0.5746 - accuracy: 0.7891 - val_loss: 1.0598 - val_accuracy: 0.3542
Epoch 7/20
5/5 [==============================] - 5s 1s/s

In [24]:
img = image.load_img('/content/drive/MyDrive/images/test/Good Personality/1.jpg', target_size=(150, 150))
labels = ['Averge Personality', 'Fair Personality', 'Good Personality']
array = image.img_to_array(img)
array = np.expand_dims(array, axis=0)
# print(array)
preds = model.predict(preprocess_input(array))
# res = decode_predictions(preds, top=1)[0]

array_preds = max(preds)
(array_preds)
index_pred = np.argmax(array_preds)
labels[index_pred]



'Good Personality'